In [ ]:
def getDualStreamModel(featCols):
    # 定义 LSTM 网络
    lstm_input = Input(shape=(None, len(featCols)))
    lstm_output1 = LSTM(256, activation='tanh', return_sequences=True)(lstm_input)
    lstm_output2 = LSTM(64, activation='tanh', return_sequences=True)(lstm_output1)
    lstm_output3 = LSTM(4, activation='tanh', return_sequences=False)(lstm_output2)

    psd_input = Input(shape=(40,))
    combined_features = Concatenate()([lstm_output3, psd_input])  # 合并特征
    flattened_features = Flatten()(combined_features)
    flattened_features_dropout = Dropout(0.2)(flattened_features)

    # 添加自注意力层
    attention_output = SelfAttention(d_model=32)(flattened_features_dropout)
    # linear ReLU
    output = Dense(1)(attention_output)

    # 创建模型
    model = Model(inputs=[lstm_input, psd_input], outputs=output)
    return model


def validatePerfErrPredictionDualStreamNetwork(valMatDF, mdl, featCols):
    subValMatDF = valMatDF
    Xz, train_steps = scaler.get_lstm_data_standard(subValMatDF, featCols)
    val_fd = valMatDF['fd']
    val_fd_numpy = val_fd.to_numpy()
    val_stacked_array = np.stack(val_fd_numpy)
    val_fd_2d = np.reshape(np.squeeze(val_stacked_array), (-1, 40))
    val_fd_input_ragged = tf.ragged.constant(val_fd_2d, dtype=np.float32)

    y = subValMatDF['cumulative_total_error'].values.astype(np.float32)

    x = tf.ragged.constant(Xz, dtype=np.float32)

    y_pred = mdl.predict([x, val_fd_input_ragged])
    y_pred = [item for sublist in y_pred for item in sublist]
    y_pred = [item for sublist in y_pred for item in sublist]

    # 皮尔逊
    pearsonCorr = scistats.pearsonr(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    # 斯皮尔曼
    [spCorr, spPval] = scistats.spearmanr(y, y_pred)
    plt.figure(figsize=(10, 8))  # 调整图表的尺寸
    plt.scatter(y, y_pred)
    lims = [min(min(y), min(y_pred)), max(max(y), max(y_pred))]
    plt.xlim(lims)
    plt.ylim(lims)
    plt.xlabel('m')
    plt.ylabel('Predicted')
    plt.title('Total Flight Error: Predicted vs. Actual\n RMSE = ' +
              str(rmse) + "\nPearson Correlation: " + str(pearsonCorr) +
              "\nSpearman Correlation: " + str(spCorr) + ", " + str(spPval) + ")")
    plt.show()
    

def plothisMse(history):
    plt.figure(figsize=(12, 6))

    # 绘制损失曲线
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # 绘制准确度曲线
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mse'], label='Training Mse')
    plt.plot(history.history['val_mse'], label='Validation Mse')
    plt.title('Mse Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Mse')
    plt.legend()

    plt.tight_layout()
    plt.show()





def validatePerfErrPredictionDualStreamNetwork(valMatDF, mdl, featCols):
    subValMatDF = valMatDF
    Xz, train_steps = scaler.get_lstm_data_standard(subValMatDF, featCols)
    val_fd = valMatDF['fd']
    val_fd_numpy = val_fd.to_numpy()
    val_stacked_array = np.stack(val_fd_numpy)
    val_fd_2d = np.reshape(np.squeeze(val_stacked_array), (-1, 40))
    val_fd_input_ragged = tf.ragged.constant(val_fd_2d, dtype=np.float32)

    y = subValMatDF['cumulative_total_error'].values.astype(np.float32)

    x = tf.ragged.constant(Xz, dtype=np.float32)

    y_pred = mdl.predict([x, val_fd_input_ragged])
    y_pred = [item for sublist in y_pred for item in sublist]
    y_pred = [item for sublist in y_pred for item in sublist]

    # 皮尔逊
    pearsonCorr = scistats.pearsonr(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    # 斯皮尔曼
    [spCorr, spPval] = scistats.spearmanr(y, y_pred)
    plt.figure(figsize=(10, 8))  # 调整图表的尺寸
    plt.scatter(y, y_pred)
    lims = [min(min(y), min(y_pred)), max(max(y), max(y_pred))]
    plt.xlim(lims)
    plt.ylim(lims)
    plt.xlabel('m')
    plt.ylabel('Predicted')
    plt.title('Total Flight Error: Predicted vs. Actual\n RMSE = ' +
              str(rmse) + "\nPearson Correlation: " + str(pearsonCorr) +
              "\nSpearman Correlation: " + str(spCorr) + ", " + str(spPval) + ")")
    plt.show()